In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from tqdm.auto import tqdm

import sys
sys.path.append("../../")
import os

import logging
from src.utils import logging_utils
from src.utils import env_utils, experiment_utils
from src import functional

logger = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.INFO,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)

import torch
import transformers

logger.info(f"{torch.__version__=}, {torch.version.cuda=}")
logger.info(f"{torch.cuda.is_available()=}, {torch.cuda.device_count()=}, {torch.cuda.get_device_name()=}")
logger.info(f"{transformers.__version__=}")

/home/local_arnab/miniconda3/envs/retrieval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2024-09-06 11:20:48 __main__ INFO     torch.__version__='2.3.1', torch.version.cuda='12.1'
2024-09-06 11:20:48 __main__ INFO     torch.cuda.is_available()=True, torch.cuda.device_count()=1, torch.cuda.get_device_name()='NVIDIA RTX A6000'
2024-09-06 11:20:48 __main__ INFO     transformers.__version__='4.44.2'


In [3]:
from dataclasses_json import DataClassJsonMixin
from dataclasses import dataclass, field, fields
from typing import Optional
import random
from src.dataset import BridgeSample, BridgeRelation, BridgeDataset
from src.dataset import load_bridge_relation, load_bridge_relations, load_bridge_dataset        

In [4]:
experiment_utils.set_seed(123456)
relations = load_bridge_relations()
dataset = BridgeDataset(relations)

2024-09-06 11:20:50 src.utils.experiment_utils INFO     setting all seeds to 123456
2024-09-06 11:20:50 src.dataset INFO     initialized bridge relation superpower_characters with 80 examples
2024-09-06 11:20:50 src.dataset INFO     initialized bridge relation sport_players with 61 examples
2024-09-06 11:20:50 src.dataset INFO     initialized bridge relation movie_actor with 71 examples
2024-09-06 11:20:50 src.dataset INFO     initialized bridge relation architect_building with 43 examples
2024-09-06 11:20:50 src.dataset INFO     initialized bridge relation none with 102 examples
2024-09-06 11:20:50 src.dataset INFO     initialized bridge dataset with 5 relations and 352 examples


In [5]:
len(dataset.relations)

5

In [6]:
sample_idx = 12
question,answer = dataset[sample_idx]

print(question)
print(answer)

Given two entities, find a common link between them.
#
What is a common link between Michelle Williams and Marilyn Monroe?
A: My Week with Marilyn - a movie where Michelle Williams played the role of Marilyn Monroe.
#
What is a common link between Fallingwater and Guggenheim Museum?
A: Frank Lloyd Wright - who was the architect of both buildings Fallingwater and Guggenheim Museum.
#
What is a common link between Roger Federer and Rafael Nadal?
A: tennis - a sport where both Roger Federer and Rafael Nadal are known for.
#
What is a common link between Charles Darwin and Flamenco?
A: none - there is no connection between Charles Darwin and Flamenco.
#
What is a common link between Mr. Fantastic and Elastigirl?
A: elastic powers - an attribute that both characters Mr. Fantastic and Elastigirl possess.
#
What is a common link between Invisible Woman and Violet Parr?
A:
force field - an attribute that both characters Invisible Woman and Violet Parr possess.


In [7]:
import torch

from nnsight import LanguageModel
from src.models import ModelandTokenizer

# model_key = "meta-llama/Meta-Llama-3-8B"
model_key = "meta-llama/Meta-Llama-3-8B-Instruct"
# model_key = "google/gemma-2-9b-it"
# model_key = "google/gemma-2-27b-it"
# model_key = "Qwen/Qwen2-7B"

mt = ModelandTokenizer(
    model_key=model_key,
    torch_dtype=torch.float16,
)

2024-09-06 11:20:54 accelerate.utils.modeling INFO     We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards: 100%|██████████| 4/4 [00:05<00:00,  1.29s/it]

2024-09-06 11:20:59 src.models INFO     loaded model </home/local_arnab/Codes/00_MODEL/meta-llama/Meta-Llama-3-8B-Instruct> | size: 15316.516 MB | dtype: torch.float16 | device: cuda:0


In [12]:
from src.functional import predict_bridge_entity

predicted_ans = predict_bridge_entity(mt, question)
predicted_ans

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


' superpowers - both Invisible Woman and Violet Parr have superpowers.\n'

In [15]:
from src.functional import verify_bridge_response

query_sample = dataset.examples[sample_idx]
verify_bridge_response(query_sample, predicted_ans)

"No. While it's true that both Invisible Woman and Violet Parr have superpowers, this is not the most specific or notable common link between them. The key connection is that they both have the power of invisibility as their primary superpower. Additionally, both characters can create force fields. These specific shared abilities are a much more precise and relevant link between the two characters than the general fact that they have superpowers."

In [16]:
from src.functional import filter_bridge_samples_by_model_knowledge

dataset = filter_bridge_samples_by_model_knowledge(mt, dataset)

  0%|          | 0/352 [00:00<?, ?it/s]

2024-09-06 00:16:10 src.functional INFO     ['Doctor Strange', 'Zatanna'] <> magic user | predicted:  magic - a theme that both Doctor Strange and Zatanna are associated with.
 => (✓)


  0%|          | 1/352 [00:04<27:21,  4.68s/it]

2024-09-06 00:16:15 src.functional INFO     ['Sonic the Hedgehog', 'The Flash'] <> super speed | predicted:  speed - a superpower that both characters Sonic the Hedgehog and The Flash possess.
 => (✓)


  1%|          | 2/352 [00:09<28:29,  4.88s/it]

...

2024-09-06 00:47:29 src.functional INFO     ['Allyson Felix', 'Jackie Joyner-Kersee'] <> athletics | predicted:  track and field - a sport where both Allyson Felix and Jackie Joyner-Kersee are known for.
 => (✓)


100%|█████████▉| 351/352 [31:23<00:05,  5.40s/it]

2024-09-06 00:47:34 src.functional INFO     ['Vision', 'Martian Manhunter'] <> density manipulation | predicted:  telepathy - a superpower that both characters Vision and Martian Manhunter possess.
 => (✗)


100%|██████████| 352/352 [31:28<00:00,  5.37s/it]

2024-09-06 00:47:34 src.functional INFO     filtered 293 samples out of 352 with 5 icl examples


In [18]:
import datetime

file_name = os.path.join(
    env_utils.DEFAULT_DATA_DIR, 
    "bridge_cached", 
    mt.name.split("/")[-1],
    f"filtered_{datetime.datetime.now().isoformat()}.json",
)

with open(file_name, "w") as f:
    f.write(dataset.to_json())

In [19]:
import json
with open(file_name, "r") as f:
    json_data = json.load(f)
loaded_dataset = BridgeDataset.from_dict(json_data)

2024-09-06 00:58:41 src.dataset INFO     initialized bridge relation superpower_characters with 47 examples
2024-09-06 00:58:41 src.dataset INFO     initialized bridge relation sport_players with 60 examples
2024-09-06 00:58:41 src.dataset INFO     initialized bridge relation movie_actor with 59 examples
2024-09-06 00:58:41 src.dataset INFO     initialized bridge relation architect_building with 36 examples
2024-09-06 00:58:41 src.dataset INFO     initialized bridge relation none with 91 examples
2024-09-06 00:58:41 src.dataset INFO     initialized bridge dataset with 5 relations and 293 examples


In [20]:
import json
import os
from tqdm.auto import tqdm

file_name = "filtered_2024-09-06T00:58:32.777195.json"
cached_known_dir = os.path.join(
    env_utils.DEFAULT_DATA_DIR, "bridge_cached", "Meta-Llama-3-8B-Instruct"
)
with open(os.path.join(cached_known_dir, file_name), "r") as f:
    json_data = json.load(f)

for relation in json_data["relations"]:
    print(f'{relation["name"]} --> {len(relation["examples"])} examples')

superpower_characters --> 47 examples
sport_players --> 60 examples
movie_actor --> 59 examples
architect_building --> 36 examples
none --> 91 examples


In [21]:
known = json_data["examples"] + ["icl_examples"]

for relation in json_data["relations"]:
    known_examples = []
    print(relation["name"])
    for examples in tqdm(relation["examples"]):
        if examples in known:
            known_examples.append(examples)
    relation["examples"] = known_examples

for relation in json_data["relations"]:
    print(f'{relation["name"]} --> {len(relation["examples"])} examples')

superpower_characters


100%|██████████| 47/47 [00:00<00:00, 218066.69it/s]


sport_players


100%|██████████| 60/60 [00:00<00:00, 286300.61it/s]


movie_actor


...

architect_building


100%|██████████| 36/36 [00:00<00:00, 267247.69it/s]


none


100%|██████████| 91/91 [00:00<00:00, 198792.53it/s]

superpower_characters --> 47 examples
sport_players --> 60 examples
movie_actor --> 59 examples
architect_building --> 36 examples
none --> 91 examples


In [20]:
import datetime

file_name = os.path.join(
    env_utils.DEFAULT_DATA_DIR, 
    "bridge_cached", 
    f"filtered_{datetime.datetime.now().isoformat()}.json"
)

with open(file_name, "w") as f:
    json.dump(json_data, f)

In [ ]:
import json


file_name = "filtered_2024-09-06T00:58:32.777195.json"
cached_known_dir = os.path.join(
    env_utils.DEFAULT_DATA_DIR, "bridge_cached", "Meta-Llama-3-8B-Instruct"
)
with open(os.path.join(cached_known_dir, file_name), "r") as f:
    json_data = json.load(f)
dataset = BridgeDataset.from_dict(json_data)

In [ ]:
from src.dataset import BridgeRelation
relation = BridgeRelation.from_dict(json_data["relations"][0])
dataset = BridgeDataset([relation])